In [1]:
import yaml
import os
import zarr
import fibsem_tools as fst
import numpy as np

In [2]:
with open("../all_data.yaml") as f:
    datas = yaml.safe_load(f)

In [ ]:
root_gt_path = "/nrs/saalfeld/heinrichl/data/cellmap_labels/fly_organelles/"

In [ ]:
def fix_offset(data, crop, orig_offset, new_offset):
    diff_offset = np.array(orig_offset) - np.array(new_offset)
    crop_root = fst.access(f"{root_gt_path}{data}/groundtruth.zarr/{crop}", mode="r+")
    for lbl in list(crop_root.keys()):
        crop_label_root = crop_root[lbl]
        ms_attrs = crop_label_root.attrs["multiscales"]
        for ds in range(len(ms_attrs[0]["datasets"])):
            for k in range(len(ms_attrs[0]["datasets"][ds]["coordinateTransformations"])):
                if ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["type"] == "translation":
                    transl = ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["translation"] 
                    ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["translation"] = list(np.array(transl) - diff_offset)
        crop_label_root.attrs["multiscales"] = ms_attrs

In [ ]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        lbl = list(crop_root.keys())[0]
        crop_datasets = crop_root[lbl].attrs["multiscales"][0]["datasets"]
        for cds in crop_datasets:
            if cds["path"] == "s0":
                crop_res = cds["coordinateTransformations"][0]["scale"]
                crop_off = cds["coordinateTransformations"][1]["translation"]
                crop_off_arr = np.array(crop_off)
                crop_res_arr = np.array(crop_res)
                corner_off_arr = (crop_off_arr + raw_res_arr/2. - crop_res_arr/2.)
                print("########################")
                if np.all(corner_off_arr % raw_res_arr == 0) and np.all(corner_off_arr % crop_res_arr == 0):
                    print(crop, "GOOD", raw_res, crop_res, crop_off)
                else:
                    print(crop, "BAD")
                    print("RAW -", raw_res, "; CROP -", crop_res, "@", crop_off,"->", corner_off_arr)
                    if np.all(crop_res < raw_res):
                        crop_off_arr_suggested = crop_off_arr - crop_res_arr/2.
                        corner_off_arr_suggested = (crop_off_arr_suggested + raw_res_arr/2. - crop_res_arr/2.)
                        if np.all(corner_off_arr_suggested % raw_res_arr == 0) and np.all(corner_off_arr_suggested % crop_res_arr == 0):
                            print("SUGGESTED WORKS:", "suggested offset:", list(crop_off_arr_suggested))
                            fix_offset(key, crop, crop_off, list(crop_off_arr_suggested))
                        else:
                            print("SUGGESTED STILL BAD")
                    else:
                        crop_off_arr_suggested = crop_off_arr + crop_res_arr/2. - raw_res_arr/2.
                        corner_off_arr_suggested = (crop_off_arr_suggested + raw_res_arr/2. - crop_res_arr/2.)
                        if np.all(corner_off_arr_suggested % raw_res_arr == 0) and np.all(corner_off_arr_suggested % crop_res_arr == 0):
                            print("SUGGESTED WORKS:", "suggested offset:", list(crop_off_arr_suggested))
                            fix_offset(key, crop, crop_off, list(crop_off_arr_suggested))
                        else:
                            print("SUGGESTED STILL BAD")
                        #print("IS DIVISIBLE: ", np.all(crop_off_arr%crop_res_arr == 0))
    print("---------------------------------------------------------------------------------------------------------------------------------------------")